## Importing the libraries:


In [2]:
import pandas as pd
from typing import Union, List, Dict
import numpy as np
import openpyxl
import re
import xlsxwriter

# Import statements for Functions.py
import sys
import os

from src.python import filter_rows_by_value, set_columns_to_datetime

sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'python')))
from Functions import *


## Importing the datasets:


In [3]:
# Importing dataset of balance and P&L function:
# Execution might take 10 minutes or more due to the large size of the datasets.
# Cannot make pandas read csv without errors so we use openpyxl to read xlsx files.

balance2025 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025.xlsx')
balance2024 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2024.xlsx')
balance2023 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2023.xlsx')
balance2022 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2022.xlsx')
balance2021 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2021.xlsx')
balance2020 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2020.xlsx')

pnl2025 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2025.xlsx')
pnl2024 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2024.xlsx')
pnl2023 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2023.xlsx')
pnl2022 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2022.xlsx')
pnl2021 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2021.xlsx')
pnl2020 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2020.xlsx')

### Fallbacks for large dfs:
Run from here when you make changes to the dataframes to avoid memory errors and long file importing!

In [4]:

# Fallbacks for large dfs:
B2025 = balance2025.copy()
B2024 = balance2024.copy()
B2023 = balance2023.copy()
B2022 = balance2022.copy()
B2021 = balance2021.copy()
B2020 = balance2020.copy()

P2025 = pnl2025.copy()
P2024 = pnl2024.copy()
P2023 = pnl2023.copy()
P2022 = pnl2022.copy()
P2021 = pnl2021.copy()
P2020 = pnl2020.copy()

# Saving in lists for functions:
balance_list = [B2025, B2024, B2023, B2022, B2021, B2020]
pnl_list = [P2025, P2024, P2023, P2022, P2021, P2020]

## Cleaning the data with functions:
#### Removing unnecessary columns:

In [5]:
# Unnecessary column removal from list of dataframes:
def remove_mutual_unnecessary_columns(df_list):
    for df in df_list:
        bad_columns = ['ja_pavadinimas', 'obj_pav','form_pav','template_name',  'standard_name', 'form_pavadinimas','line_type_id', 'stat_pavadinimas', 'stat_pav']
        for col in bad_columns:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
    return df_list

# Removing unnecessary rows from list of dataframes:
remove_mutual_unnecessary_columns(balance_list)
remove_mutual_unnecessary_columns(pnl_list)

# Renaming collumns to be the same across dataframes:
def rename_columns(df_list):
    for df in df_list:
        # Rename 'obj_kodas' to 'ja_kodas' if it exists
        if 'obj_kodas' in df.columns:
            df.rename(columns={'obj_kodas': 'ja_kodas'}, inplace=True)

        # Rename other columns if they exist
        column_mapping = {
            'nuosavas_kapitalas': 'NUOSAVAS KAPITALAS',
            'mok_sumos_ir_isipareigojimai': 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI',
            'trumpalaikis_turtas': 'TRUMPALAIKIS TURTAS',
            'ilgalaikis_turtas': 'ILGALAIKIS TURTAS',
            'pelnas_pries_apmokestinima': 'PELNAS (NUOSTOLIAI) PRIEŠ APMOKESTINIMĄ',
        'grynasis_pelnas': 'GRYNASIS PELNAS (NUOSTOLIAI)',
        'pardavimo_pajamos': 'PARDAVIMO PAJAMOS'
        }

        for old_col, new_col in column_mapping.items():
            if old_col in df.columns:
                df.rename(columns={old_col: new_col}, inplace=True)

    return df_list

rename_columns(balance_list)
rename_columns(pnl_list)




[         ja_kodas  form_kodas  stat_kodas template_id standard_id  \
 0       110003978         310           0      FS0329      IST024   
 1       110003978         310           0      FS0329      IST024   
 2       110003978         310           0      FS0329      IST024   
 3       110004884         310           0      FS0718      IST209   
 4       110004884         310           0      FS0718      IST209   
 ...           ...         ...         ...         ...         ...   
 423454  307123738         310           0      FS0329      IST024   
 423455  307193537         960           0      FS0522      IST024   
 423456  307193537         960           0      FS0522      IST024   
 423457  307438075         960           0      FS0522      IST118   
 423458  307438075         960           0      FS0522      IST118   
 
                                       line_name  reiksme beginning_date  \
 0                             PARDAVIMO PAJAMOS    97545     2024-01-01   
 1    

#### Extracting columns line_name, reiksme and ja_kodas:
This function is uneeded but if you need to view just the extracted columns, you can use it:

In [6]:
# Extracting columns line_name, reiksme and ja_kodas from dfs with all of this data:
def extract_line_name_reiksme_ja_kodas(df_list):
    """
    Extract columns 'line_name', 'reiksme', and 'ja_kodas' from DataFrames
    that contain all three columns.

    Parameters:
    -----------
    df_list : list of pandas.DataFrame
        List of DataFrames to process (will be modified in-place)

    Returns:
    --------
    list of pandas.DataFrame
        List of DataFrames containing only the three specified columns
    """
    extracted_dfs = []

    required_columns = ['line_name', 'reiksme', 'ja_kodas']

    for i, df in enumerate(df_list):
        # Check if all required columns exist in the current DataFrame
        if all(col in df.columns for col in required_columns):
            # Extract only the required columns
            extracted_df = df[required_columns].copy()
            extracted_dfs.append(extracted_df)


        else:
            missing_cols = [col for col in required_columns if col not in df.columns]
            print(f"DataFrame {i}: Missing columns {missing_cols} - skipped")

    print(f"\nExtracted {len(extracted_dfs)} out of {len(df_list)} DataFrames")
    return extracted_dfs
""" Uncomment to see the extracted columns:
B_extracted = extract_line_name_reiksme_ja_kodas(balance_list)
P_extracted = extract_line_name_reiksme_ja_kodas(pnl_list)

# Renaming the extracted dfs to be more descriptive:
B_extracted_2025 = B_extracted[0]
B_extracted_2024 = B_extracted[1]

P_extracted_2025 = P_extracted[0]
P_extracted_2024 = P_extracted[1]

# New lists with extracted and renamed dfs:
B_extracted_renamed = [B_extracted_2025, B_extracted_2024]
P_extracted_renamed = [P_extracted_2025, P_extracted_2024]
"""

' Uncomment to see the extracted columns:\nB_extracted = extract_line_name_reiksme_ja_kodas(balance_list)\nP_extracted = extract_line_name_reiksme_ja_kodas(pnl_list)\n\n# Renaming the extracted dfs to be more descriptive:\nB_extracted_2025 = B_extracted[0]\nB_extracted_2024 = B_extracted[1]\n\nP_extracted_2025 = P_extracted[0]\nP_extracted_2024 = P_extracted[1]\n\n# New lists with extracted and renamed dfs:\nB_extracted_renamed = [B_extracted_2025, B_extracted_2024]\nP_extracted_renamed = [P_extracted_2025, P_extracted_2024]\n'

#### Shifting the column data of extraced dfs so that every row is unique with new columns:

In [8]:

# Pivoting the balance dataframes:

Bpivoted = pivot_dfs_smart(balance_list, date_column='reg_date')

# Pivoting the P&L dataframes:

Ppivoted = pivot_dfs_smart(pnl_list, date_column='reg_date')



DataFrame 0: Preserving 9 columns in result


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



DataFrame 0: Successfully pivoted. Original shape: (615188, 11), Pivoted shape: (149717, 18)
Removed 19539 duplicate rows
DataFrame 1: Preserving 9 columns in result


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



 - Keeping most recent row (index 541841)
 - Removing 1 duplicates (indices: [541837])
 - Differing reiksme values: [588615, 2147]

Duplicate Group: ja_kodas=306131408, line_name=NUOSAVAS KAPITALAS
 - Keeping most recent row (index 541840)
 - Removing 1 duplicates (indices: [541836])
 - Differing reiksme values: [113651, 34115]

Duplicate Group: ja_kodas=306131408, line_name=TRUMPALAIKIS TURTAS
 - Keeping most recent row (index 541839)
 - Removing 1 duplicates (indices: [541835])
 - Differing reiksme values: [702266, 36262]

Duplicate Group: ja_kodas=306132346, line_name=ILGALAIKIS TURTAS
 - Keeping most recent row (index 542029)
 - Removing 1 duplicates (indices: [542028])
 - Differing reiksme values: [16219, 16844]

Duplicate Group: ja_kodas=306132346, line_name=MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI
 - Keeping most recent row (index 542035)
 - Removing 1 duplicates (indices: [542034])
 - Differing reiksme values: [20113, 18625]

Duplicate Group: ja_kodas=306132346, line_name=NUOSAV

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



DataFrame 1: Successfully pivoted. Original shape: (406755, 11), Pivoted shape: (144312, 17)
Removed 23856 duplicate rows
DataFrame 2: Missing columns ['line_name', 'reiksme']
DataFrame 3: Missing columns ['line_name', 'reiksme']
DataFrame 4: Missing columns ['line_name', 'reiksme']
DataFrame 5: Missing columns ['line_name', 'reiksme']

🎯 FINAL DUPLICATE REMOVAL SUMMARY:
   📊 Total duplicate rows removed across all DataFrames: 38574
   📅 Using date column 'reg_date' - kept most recent rows in duplicate groups


### Checking duplicate rows in all dfs:

This removes duplicate rows from all dfs and keeps only the most recent row for each ja_kodas, it keeps the new revision data. Older revisions are extracted and saved in a separate df.

In [15]:
# Making some columns date format:

datecols = ['reg_date', 'beginning_date', 'turning_date']

set_columns_to_datetime(Bpivoted, datecols)
set_columns_to_datetime(Ppivoted, datecols)

#Removing non year financial year data: from each df, we remove all data except for the respective financial year.

#only_fullyear_2025 = filter_rows_by_value(Ppivoted[0], ['beginning_date', 'turning_date'] , ['2024-01-01', '2025-12-31'])


KeyError: 'beginning_date'

## Adding PnL data to balance data:


In [14]:
def merge_pnl_and_balances(df_list1, df_list2, how='inner'):
    """
    Merge two lists of DataFrames on ja_kodas column with comprehensive diagnostics and validation.
    Each DataFrame from list1 is merged with corresponding DataFrame from list2.

    Parameters:
    -----------
    df_list1, df_list2 : list of pandas.DataFrame
        Lists of DataFrames to merge
    how : str, default 'inner'
        Type of merge: 'inner', 'left', 'right', 'outer'

    Returns:
    --------
    list of pandas.DataFrame
        List of merged DataFrames
    """
    if len(df_list1) != len(df_list2):
        print(f"Warning: List lengths differ - list1: {len(df_list1)}, list2: {len(df_list2)}")
        # Use the minimum length to avoid index errors
        min_length = min(len(df_list1), len(df_list2))
        df_list1 = df_list1[:min_length]
        df_list2 = df_list2[:min_length]

    merged_dfs = []

    for i, (df1, df2) in enumerate(zip(df_list1, df_list2)):
        try:
            # Check if ja_kodas exists in both DataFrames
            if 'ja_kodas' not in df1.columns:
                print(f"Pair {i}: ja_kodas not found in first DataFrame, skipping")
                continue
            if 'ja_kodas' not in df2.columns:
                print(f"Pair {i}: ja_kodas not found in second DataFrame, skipping")
                continue

            # Create copies to avoid modifying originals
            df1_clean = df1.copy()
            df2_clean = df2.copy()

            # Validate ja_kodas data types and convert if necessary
            if df1_clean['ja_kodas'].dtype != df2_clean['ja_kodas'].dtype:
                print(f"Pair {i}: ja_kodas data types differ - converting both to string")
                df1_clean['ja_kodas'] = df1_clean['ja_kodas'].astype(str)
                df2_clean['ja_kodas'] = df2_clean['ja_kodas'].astype(str)

            # Check for duplicate ja_kodas within each DataFrame
            df1_duplicates = df1_clean.duplicated(subset=['ja_kodas']).sum()
            df2_duplicates = df2_clean.duplicated(subset=['ja_kodas']).sum()

            if df1_duplicates > 0:
                print(f"Pair {i}: WARNING - {df1_duplicates} duplicate ja_kodas found in first DataFrame")
                # Keep first occurrence of duplicates
                df1_clean = df1_clean.drop_duplicates(subset=['ja_kodas'], keep='first')

            if df2_duplicates > 0:
                print(f"Pair {i}: WARNING - {df2_duplicates} duplicate ja_kodas found in second DataFrame")
                # Keep first occurrence of duplicates
                df2_clean = df2_clean.drop_duplicates(subset=['ja_kodas'], keep='first')

            # Check for NaN values in ja_kodas
            df1_nan = df1_clean['ja_kodas'].isna().sum()
            df2_nan = df2_clean['ja_kodas'].isna().sum()

            if df1_nan > 0:
                print(f"Pair {i}: WARNING - {df1_nan} NaN values in ja_kodas (first DataFrame), removing")
                df1_clean = df1_clean.dropna(subset=['ja_kodas'])

            if df2_nan > 0:
                print(f"Pair {i}: WARNING - {df2_nan} NaN values in ja_kodas (second DataFrame), removing")
                df2_clean = df2_clean.dropna(subset=['ja_kodas'])

            # Pre-merge diagnostics
            df1_unique = df1_clean['ja_kodas'].nunique()
            df2_unique = df2_clean['ja_kodas'].nunique()
            common_ja_kodas = set(df1_clean['ja_kodas']) & set(df2_clean['ja_kodas'])
            common_count = len(common_ja_kodas)

            print(f"\n--- Pair {i} Merge Diagnostics ---")
            print(f"DF1: {len(df1_clean)} rows, {df1_unique} unique ja_kodas")
            print(f"DF2: {len(df2_clean)} rows, {df2_unique} unique ja_kodas")
            print(f"Common ja_kodas: {common_count}")
            print(f"Merge type: {how}")

            # Calculate expected result sizes
            if how == 'inner':
                expected_rows = common_count
            elif how == 'left':
                expected_rows = len(df1_clean)
            elif how == 'right':
                expected_rows = len(df2_clean)
            else:  # outer
                expected_rows = len(df1_clean) + len(df2_clean) - common_count

            print(f"Expected result rows: {expected_rows}")

            # Perform the merge with indicator to track sources
            merged_df = pd.merge(
                df1_clean,
                df2_clean,
                on='ja_kodas',
                how=how,
                suffixes=('_pnl', '_balance'),
                indicator=True  # Add merge indicator column
            )

            # Post-merge diagnostics
            actual_rows = len(merged_df)
            merge_stats = merged_df['_merge'].value_counts()

            print(f"Actual result rows: {actual_rows}")
            print(f"Merge composition: {merge_stats.to_dict()}")

            if actual_rows != expected_rows:
                print(f"WARNING: Expected {expected_rows} rows but got {actual_rows} rows")

            # Remove the indicator column
            merged_df = merged_df.drop('_merge', axis=1)

            # Check for overlapping column names (besides ja_kodas)
            overlapping_cols = set(df1_clean.columns) & set(df2_clean.columns) - {'ja_kodas'}
            if overlapping_cols:
                print(f"Overlapping columns (received suffixes): {list(overlapping_cols)}")

            print(f"Pair {i}: Successfully merged. Shapes: {df1.shape} + {df2.shape} -> {merged_df.shape}")

            merged_dfs.append(merged_df)

        except Exception as e:
            print(f"Pair {i}: Error during merge - {e}")
            print(f"Pair {i}: DF1 columns: {list(df1.columns) if 'df1' in locals() else 'N/A'}")
            print(f"Pair {i}: DF2 columns: {list(df2.columns) if 'df2' in locals() else 'N/A'}")
            # Keep both original DataFrames if merge fails
            merged_dfs.extend([df1, df2])

    # Final summary
    print(f"\n=== MERGE SUMMARY ===")
    print(f"Successfully processed: {len(merged_dfs)} DataFrames")
    print(f"Total input pairs: {min(len(df_list1), len(df_list2))}")

    return merged_dfs

# Merge the cleaned balance and P&L data:
MergedData = merge_pnl_and_balances(cleanedB, cleanedP)

# Merging the cleaned balance and P&L data but with outer merge (all rows are kept):
# Seems to be the same as inner merge, I don't know why.
MergedDataAll = merge_pnl_and_balances(cleanedB, cleanedP, how='outer')

# Renaming dfs in MergedData to be more descriptive:

MergedBP2025 = MergedData[0]
MergedBP2024 = MergedData[1]
MergedBP2023 = MergedData[2]
MergedBP2022 = MergedData[3]
MergedBP2021 = MergedData[4]
MergedBP2020 = MergedData[5]

MergedBPall2025 = MergedDataAll[0]
MergedBPall2024 = MergedDataAll[1]
MergedBPall2023 = MergedDataAll[2]
MergedBPall2022 = MergedDataAll[3]
MergedBPall2021 = MergedDataAll[4]
MergedBPall2020 = MergedDataAll[5]



Pair 0: WARNING - 138901 duplicate ja_kodas found in first DataFrame
Pair 0: WARNING - 138901 duplicate ja_kodas found in second DataFrame

--- Pair 0 Merge Diagnostics ---
DF1: 146512 rows, 146512 unique ja_kodas
DF2: 146512 rows, 146512 unique ja_kodas
Common ja_kodas: 146512
Merge type: inner
Expected result rows: 146512
Actual result rows: 146512
Merge composition: {'both': 146512, 'left_only': 0, 'right_only': 0}
Overlapping columns (received suffixes): ['formavimo_data', 'standard_id', 'stat_kodas', 'beginning_date', 'template_id', 'turning_date', 'form_kodas', 'reg_date']
Pair 0: Successfully merged. Shapes: (285413, 18) + (285413, 17) -> (146512, 34)
Pair 1: WARNING - 132026 duplicate ja_kodas found in first DataFrame
Pair 1: WARNING - 132026 duplicate ja_kodas found in second DataFrame

--- Pair 1 Merge Diagnostics ---
DF1: 141315 rows, 141315 unique ja_kodas
DF2: 141315 rows, 141315 unique ja_kodas
Common ja_kodas: 141315
Merge type: inner
Expected result rows: 141315
Actual 

### Data validation:

In [26]:
#describe_dataframes(MergedData)

describe_dataframes(MergedDataAll)

,dataframe_name,rows,columns,total_cells,memory_mb,ja_kodas_column_exists,ja_kodas_duplicates,ja_kodas_total_duplicate_rows,ja_kodas_unique_duplicate_values,ja_kodas_duplicate_percentage,ja_kodas_most_common_duplicate,ja_kodas_most_common_count,total_missing_values,missing_percentage,numeric_columns,categorical_columns,unique_dtypes
0,df_0,146512,34,4981408,107.03,True,0,0,0,0.0,None,0,1513255,30.38,22,10,4
1,df_1,141315,34,4804710,103.23,True,0,0,0,0.0,None,0,1469938,30.59,22,10,4
2,df_2,151619,24,3638856,95.30,True,0,0,0,0.0,None,0,456253,12.54,12,10,4
3,df_3,119072,24,2857728,76.51,True,0,0,0,0.0,None,0,207649,7.27,12,10,4
4,df_4,106775,24,2562600,68.79,True,0,0,0,0.0,None,0,165737,6.47,12,10,4
5,df_5,96352,24,2312448,62.05,True,0,0,0,0.0,None,0,147251,6.37,12,10,4


### Interim data export:
Notebook is getting too long and confusing, so I will export the interim data to csv files.

In [24]:
# Exporting the interim data to csv files:

save_df_list_to_csv_auto(MergedData, '../../data/interim/joined-balance-and-PnL/joined-only-matching-ja-kodas')

save_df_list_to_csv_auto(MergedDataAll, '../../data/interim/joined-balance-and-PnL/joined-all-ja-kodas')

['MergedBPall2025',
 'MergedBPall2024',
 'MergedBPall2023',
 'MergedBPall2022',
 'MergedBPall2021',
 'MergedBPall2020']